<a href="https://colab.research.google.com/github/WalterPHD/Ai-Data/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 1

SimpleRNN, GRU, LSTM on IMDB

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Load IMDB dataset
max_features = 20000  # number of words
maxlen = 200          # cut reviews after 200 words

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

def build_model(rnn_layer):
    model = keras.Sequential([
        layers.Embedding(max_features, 128),
        rnn_layer,
        layers.Dense(1, activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

results = {}

# Try different recurrent layers
for rnn_layer in [
    layers.SimpleRNN(64),
    layers.GRU(64),
    layers.LSTM(64),
]:
    print(f"\nTraining with {rnn_layer.__class__.__name__}...")
    model = build_model(rnn_layer)
    history = model.fit(
        x_train, y_train,
        batch_size=64,
        epochs=2,          # keep small for runtime
        validation_split=0.2,
        verbose=2
    )
    score = model.evaluate(x_test, y_test, verbose=0)
    results[rnn_layer.__class__.__name__] = score[1]

print("\nFinal Accuracy Comparison:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

Training with SimpleRNN...
Epoch 1/2
313/313 - 35s - 112ms/step - accuracy: 0.6901 - loss: 0.5705 - val_accuracy: 0.7362 - val_loss: 0.5276
Epoch 2/2
313/313 - 34s - 109ms/step - accuracy: 0.8654 - loss: 0.3243 - val_accuracy: 0.7086 - val_loss: 0.5846

Training with GRU...
Epoch 1/2
313/313 - 98s - 314ms/step - accuracy: 0.7439 - loss: 0.4938 - val_accuracy: 0.8468 - val_loss: 0.3523
Epoch 2/2
313/313 - 102s - 326ms/step - accuracy: 0.8986 - loss: 0.2598 - val_accuracy: 0.8476 - val_loss: 0.3700

Training with LSTM...
Epoch 1/2
313/313 - 92s - 293ms/step - accuracy: 0.7892 - loss: 0.4444 - val_accuracy: 0.8466 - val_loss: 0.3595
Epoch 2/2
313/313 - 93s - 298ms/step - accuracy: 0.9075 - loss: 0.2406 - val_accuracy: 0.8106 - val_loss: 0.3978

Final Accuracy Comparison:
SimpleRNN: 0.7134
GRU: 0.8361
LSTM: 0.8135


ConvLSTM2D

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv3D

# Example ConvLSTM2D model
model = Sequential([
    ConvLSTM2D(filters=40, kernel_size=(3, 3), input_shape=(None, 40, 40, 1), padding="same", return_sequences=True),
    BatchNormalization(),
    ConvLSTM2D(filters=40, kernel_size=(3, 3), padding="same", return_sequences=True),
    BatchNormalization(),
    Conv3D(filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same")
])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, None, 40, 40,   │        59,200 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 40, 40,   │           160 │
│ (BatchNormalization)            │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, None, 40, 40,   │       115,360 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 40, 40,   │           160 │
│ (BatchNormalization)            │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, None, 40, 40,   │         1,081 │
│                                 │ 1)                     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 175,961 (687.35 KB)

 Trainable params: 175,801 (686.72 KB)

 Non-trainable params: 160 (640.00 B)

Problem 2

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters
max_features = 10000  # Number of words to keep
maxlen = 500          # Cut texts after 500 words

# Load Reuters dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.reuters.load_data(num_words=max_features)

# Pad sequences so all have same length
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

# Convert labels to categorical
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
def build_lstm():
    model = keras.Sequential([ layers.Embedding(max_features, 128, input_length=maxlen), layers.LSTM(64), layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(loss="categorical_crossentropy", optimizer="adam",  metrics=["accuracy"])
    return model


In [5]:
def build_conv1d():
    model = keras.Sequential([
        layers.Embedding(max_features, 128, input_length=maxlen),
        layers.Conv1D(64, 5, activation="relu"),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(64, 5, activation="relu"),
        layers.GlobalMaxPooling1D(),
        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model


In [6]:
results = {}

# Train LSTM
print("\nTraining LSTM...")
lstm = build_lstm()
lstm.fit(x_train, y_train,  batch_size=64, epochs=3,  validation_split=0.2, verbose=2)
score = lstm.evaluate(x_test, y_test, verbose=0)
results["LSTM"] = score[1]

# Train Conv1D
print("\nTraining Conv1D...")
conv = build_conv1d()
conv.fit(x_train, y_train, batch_size=64, epochs=3, validation_split=0.2, verbose=2)
score = conv.evaluate(x_test, y_test, verbose=0)
results["Conv1D"] = score[1]

print("\nFinal Test Accuracy Comparison:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")



Training LSTM...
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


113/113 - 86s - 761ms/step - accuracy: 0.4348 - loss: 2.3859 - val_accuracy: 0.4925 - val_loss: 2.0147
Epoch 2/3
113/113 - 79s - 696ms/step - accuracy: 0.4767 - loss: 2.0561 - val_accuracy: 0.2298 - val_loss: 2.2125
Epoch 3/3
113/113 - 81s - 720ms/step - accuracy: 0.5228 - loss: 1.8136 - val_accuracy: 0.5665 - val_loss: 1.7068

Training Conv1D...
Epoch 1/3
113/113 - 37s - 324ms/step - accuracy: 0.4575 - loss: 2.2309 - val_accuracy: 0.5554 - val_loss: 1.7245
Epoch 2/3
113/113 - 34s - 304ms/step - accuracy: 0.6092 - loss: 1.5819 - val_accuracy: 0.6377 - val_loss: 1.4780
Epoch 3/3
113/113 - 41s - 364ms/step - accuracy: 0.7029 - loss: 1.2665 - val_accuracy: 0.6962 - val_loss: 1.3075

Final Test Accuracy Comparison:
LSTM: 0.5637
Conv1D: 0.6803


Problem 3

In [11]:
import os
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

# Download dataset
url = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
fname = keras.utils.get_file("jena_climate.csv.zip", url, extract=True)

# Construct the correct file path within the extracted directory
fpath = os.path.join(os.path.dirname(fname), os.path.splitext(os.path.basename(fname))[0], "jena_climate_2009_2016.csv")

# Load into dataframe
df = pd.read_csv(fpath)

print(df.head())

             Date Time  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  rh (%)  \
0  01.01.2009 00:10:00    996.52     -8.02    265.40        -8.90    93.3   
1  01.01.2009 00:20:00    996.57     -8.41    265.01        -9.28    93.4   
2  01.01.2009 00:30:00    996.53     -8.51    264.91        -9.31    93.9   
3  01.01.2009 00:40:00    996.51     -8.31    265.12        -9.07    94.2   
4  01.01.2009 00:50:00    996.51     -8.27    265.15        -9.04    94.1   

   VPmax (mbar)  VPact (mbar)  VPdef (mbar)  sh (g/kg)  H2OC (mmol/mol)  \
0          3.33          3.11          0.22       1.94             3.12   
1          3.23          3.02          0.21       1.89             3.03   
2          3.21          3.01          0.20       1.88             3.02   
3          3.26          3.07          0.19       1.92             3.08   
4          3.27          3.08          0.19       1.92             3.09   

   rho (g/m**3)  wv (m/s)  max. wv (m/s)  wd (deg)  
0       1307.75      1.03        

In [13]:
# Convert dataframe to numpy (excluding the 'Date Time' column)
data = df.iloc[:, 1:].values

# Standardize (normalize)
mean = data[:200000].mean(axis=0)
std = data[:200000].std(axis=0)
data = (data - mean) / std

In [14]:
# Parameters
lookback = 720     # how many past timesteps (10 min * 720 = 5 days)
step = 6           # sample one data point every hour
delay = 144        # predict 24 hours in the future
batch_size = 128

def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while True:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]  # column 1 = temp in dataset
        yield samples, targets


In [19]:
train_gen = generator(data, lookback=lookback, delay=delay, min_index=0, max_index=200000, shuffle=True, step=step, batch_size=batch_size)

val_gen = generator(data, lookback=lookback, delay=delay, min_index=200001, max_index=300000, step=step, batch_size=batch_size)

test_gen = generator(data, lookback=lookback, delay=delay, min_index=300001, max_index=None, step=step, batch_size=batch_size)

val_steps = (300000 - 200001 - lookback) // batch_size
test_steps = (len(data) - 300001 - lookback) // batch_size


In [20]:
model = keras.Sequential([layers.LSTM(32, input_shape=(lookback // step, data.shape[-1])), layers.Dense(1)
])

model.compile(optimizer="rmsprop", loss="mae")


In [17]:
history = model.fit(train_gen, steps_per_epoch=500, epochs=20, validation_data=val_gen, validation_steps=val_steps)


Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 62s 118ms/step - loss: 0.3321 - val_loss: 0.2817
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 84s 168ms/step - loss: 0.2791 - val_loss: 0.2697
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 79s 157ms/step - loss: 0.2655 - val_loss: 0.2787
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 83s 166ms/step - loss: 0.2551 - val_loss: 0.2824
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 53s 107ms/step - loss: 0.2458 - val_loss: 0.2810
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 53s 106ms/step - loss: 0.2404 - val_loss: 0.2852
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 164ms/step - loss: 0.2335 - val_loss: 0.2888
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 53s 106ms/step - loss: 0.2298 - val_loss: 0.2909
Epoch 9/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 53s 106ms/step - loss: 0.2228 - val_loss: 0.2950
Epoch 10/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 81s 163ms/step - loss: 0.2196 - val_loss: 0.2983
Epoch 11/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 54s 108ms/step - loss: 0.2126 - val_loss: 0.2999
Epoch 12/20
500/500

In [18]:
test_loss = model.evaluate(test_gen, steps=test_steps)
print("Test MAE:", test_loss)


936/936 ━━━━━━━━━━━━━━━━━━━━ 28s 30ms/step - loss: 0.3295
Test MAE: 0.32906925678253174
